In [119]:
%%writefile ../../Geolife_Trajectories_1.3/toy_data/001.csv
001,39.984686,116.318417,492,2008-10-23,01:53:15,NA
001,39.984688,116.318385,492,2008-10-23,02:53:20,taxi
001,39.984606,116.317065,505,2008-10-23,02:53:50,taxi

Overwriting ../../Geolife_Trajectories_1.3/toy_data/001.csv


In [118]:
%%writefile ../../Geolife_Trajectories_1.3/toy_data/002.csv
002,39.984655,116.318263,492,2008-10-23,02:51:25,taxi
002,39.984539,116.317294,500,2008-10-23,02:53:15,NA
002,39.984606,116.317065,505,2008-10-23,02:53:50,taxi

Writing ../../Geolife_Trajectories_1.3/toy_data/002.csv


In [7]:
!aws s3 cp --recursive ../../Geolife_Trajectories_1.3/data_csv s3://w209-d3/data_csv

upload: ../../Geolife_Trajectories_1.3/data_csv/001.csv to s3://w209-d3/data_csv/001.csv
upload: ../../Geolife_Trajectories_1.3/data_csv/000.csv to s3://w209-d3/data_csv/000.csv
upload: ../../Geolife_Trajectories_1.3/data_csv/005.csv to s3://w209-d3/data_csv/005.csv
upload: ../../Geolife_Trajectories_1.3/data_csv/006.csv to s3://w209-d3/data_csv/006.csv
upload: ../../Geolife_Trajectories_1.3/data_csv/002.csv to s3://w209-d3/data_csv/002.csv
upload: ../../Geolife_Trajectories_1.3/data_csv/004.csv to s3://w209-d3/data_csv/004.csv
upload: ../../Geolife_Trajectories_1.3/data_csv/008.csv to s3://w209-d3/data_csv/008.csv
upload: ../../Geolife_Trajectories_1.3/data_csv/009.csv to s3://w209-d3/data_csv/009.csv
upload: ../../Geolife_Trajectories_1.3/data_csv/007.csv to s3://w209-d3/data_csv/007.csv
upload: ../../Geolife_Trajectories_1.3/data_csv/003.csv to s3://w209-d3/data_csv/003.csv
upload: ../../Geolife_Trajectories_1.3/data_csv/012.csv to s3://w209-d3/data_csv/012.csv
upload: ../../Geolife

In [8]:
!aws s3 cp --recursive ../../Geolife_Trajectories_1.3/data_csv_test s3://w209-d3/data_csv_test

upload: ../../Geolife_Trajectories_1.3/data_csv_test/000.csv to s3://w209-d3/data_csv_test/000.csv
upload: ../../Geolife_Trajectories_1.3/data_csv_test/001.csv to s3://w209-d3/data_csv_test/001.csv


In [15]:
!head -n1000 ../../Geolife_Trajectories_1.3/data_csv/000.csv > ../../Geolife_Trajectories_1.3/data_csv_test_short/000.csv
!head -n1000 ../../Geolife_Trajectories_1.3/data_csv/001.csv > ../../Geolife_Trajectories_1.3/data_csv_test_short/001.csv

In [16]:
!aws s3 cp --recursive ../../Geolife_Trajectories_1.3/data_csv_test_short s3://w209-d3/data_csv_test_short

upload: ../../Geolife_Trajectories_1.3/data_csv_test_short/001.csv to s3://w209-d3/data_csv_test_short/001.csv
upload: ../../Geolife_Trajectories_1.3/data_csv_test_short/000.csv to s3://w209-d3/data_csv_test_short/000.csv


In [76]:
from datetime import datetime

date,time=["2008-10-23", "02:53:04"]
dt = date+' '+time
a = datetime.strptime(dt, '%Y-%m-%d %H:%M:%S')
b = datetime(2008, 10, 23, 2, 53, 9)

time_diff=b-a

time_diff.total_seconds() < 6


True

#Calculate Distance

In [177]:
%%writefile CalcDist.py

from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.compat import jobconf_from_env
from datetime import datetime
import math
import ast

sort_conf = {
    'mapred.reduce.tasks': '1'
}

#convert coordinates to distance
def coor_to_km(x1,y1,x2,y2):
    R = 6371
    dx = (x2-x1) * math.pi / 180
    dy = (y2-y1) * math.pi / 180
    x1 *= math.pi / 180
    x2 *= math.pi / 180
    
    a = math.sin(dx/2)**2 + (math.sin(dy/2)**2) * (math.cos(x1)*math.cos(x2))
    c = 2*math.atan2(math.sqrt(a),math.sqrt(1-a))
    d = R * c
    return d

#convert to seconds (since python 2.6 or below does not have datetime.total_seconds()
#http://stackoverflow.com/questions/28089558/alternative-to-total-seconds-in-python-2-6
def timedelta_total_seconds(timedelta):
    return (
        timedelta.microseconds + 0.0 +
        (timedelta.seconds + timedelta.days * 24 * 3600) * 10 ** 6) / 10 ** 6


class CalcDist(MRJob):
    def steps(self):
        return [MRStep(mapper=self.mapper,
                       reducer_init=self.reducer_init,
                       reducer=self.reducer,
                       jobconf=sort_conf)]
    
    def mapper(self, _, line):
        user_id,x,y,alt,date,time,transport = line.strip().split(",")
        dt = date+' '+time
        yield (user_id,dt), (x,y,transport)
    

    def reducer_init(self):
        self.dt_initial = datetime(2008, 1, 1, 0, 0, 0)
        self.x_initial=0.0
        self.y_initial=0.0
        self.user = ''
        self.dist = {'all':0.0}
        self.max_interval_sec = int(jobconf_from_env('max_interval_sec'))

    
    def reducer(self, key, values):
        
        index=0
        user_id,dt = key
        dt_initial = datetime(2008, 1, 1, 0, 0, 0)
        dt_key = datetime.strptime(dt, '%Y-%m-%d %H:%M:%S')
        time_diff = dt_key - self.dt_initial
#         time_diff = time_diff.total_seconds()  #only for python 2.7 or above
        time_diff = timedelta_total_seconds(time_diff)
        self.dt_initial = dt_key
        
        for value in values:
            x_new,y_new,transport = value
            x_new=float(x_new)
            y_new=float(y_new)
            
            #assume only within certain time interval belong to be in the same trip
            if time_diff < self.max_interval_sec:
                distance = coor_to_km(self.x_initial,self.y_initial,x_new,y_new)
                self.dist['all'] += distance
                self.dist.setdefault(transport,0.0)
                self.dist[transport] += distance

            #if different user, reset distance counter
            if self.user != user_id:
                self.user = user_id
                self.dist = {'all':0.0}
                

            self.x_initial = x_new
            self.y_initial = y_new
            
            yield None, (user_id,dt,x_new,y_new,transport,self.dist)
            
if __name__ == '__main__':
    CalcDist.run()   
                

Overwriting CalcDist.py


In [96]:
#create new job flow
!python /Users/maktrix/Dropbox/Berkeley/W261_ML_scale/mrjob/mrjob/tools/emr/create_job_flow.py

using configs in /Users/maktrix/.mrjob.conf
Creating persistent job flow to run several jobs in...
creating tmp directory /var/folders/s3/ll81spln3fn418tf8hhv248m0000gn/T/no_script.maktrix.20151108.105200.610338
writing master bootstrap script to /var/folders/s3/ll81spln3fn418tf8hhv248m0000gn/T/no_script.maktrix.20151108.105200.610338/b.py
Copying non-input files into s3://w209-d3/no_script.maktrix.20151108.105200.610338/files/
Waiting 5.0s for S3 eventual consistency
Creating Elastic MapReduce job flow
Job flow created with ID: j-1GNVZZQS3Z1VV
j-1GNVZZQS3Z1VV


In [180]:
%reload_ext autoreload
%autoreload 2

import os

def mrjob_driver(inputfolder='', outputfolder='', onEMR=False, job_flow_id=''):
    
    from CalcDist import CalcDist

    if onEMR:
        os.system("aws s3 rm --recursive "+outputfolder)
        mr_job = CalcDist(args=[inputfolder,
                                '-r','emr',
#                                 '--emr-job-flow-id=' + job_flow_id,
                                '--output-dir=' + outputfolder,
                                '--jobconf', 'max_interval_sec=1800'])

    else: 
        !rm -r ../../Geolife_Trajectories_1.3/local/calc-dist/
        mr_job = CalcDist(args=[inputfolder,
                                '--output-dir=' + '../../Geolife_Trajectories_1.3/local/calc-dist/',
                                '--jobconf', 'max_interval_sec=1800'])

    with mr_job.make_runner() as runner:
        runner.run()

# run driver function (locally)
# mrjob_driver(inputfolder='../../Geolife_Trajectories_1.3/toy_data')       

#run driver function (EMR)
mrjob_driver(inputfolder='s3://w209-d3/data_csv/', 
             outputfolder='s3://w209-d3/calc-dist2/data_csv/',
             onEMR=True 
#              job_flow_id='j-1GNVZZQS3Z1VV'
            )


In [179]:
##EMR RESULTS
import smart_open
index = 0
filename='s3://w209-d3/calc-dist2/data_csv/part-00000'
for line in smart_open.smart_open(filename):
    print line.strip().split("\t")
    index +=1
    if index==10000: break

##LOCAL RESULTS  
# !cat ../../Geolife_Trajectories_1.3/local/calc-dist/*
# !head -n1000 ../../Geolife_Trajectories_1.3/local/calc-dist/*
# !tail -n50 ../../Geolife_Trajectories_1.3/local/calc-dist/*

null	["000", "2008-10-23 02:53:04", 39.984702, 116.318417, "NA", {"all": 0.0}]
null	["000", "2008-10-23 02:53:10", 39.984683, 116.31845, "bike", {"all": 0.003516886428136976, "bike": 0.003516886428136976}]
null	["000", "2008-10-23 02:53:52", 39.984683, 116.31845, "taxi", {"taxi": 0.0, "all": 0.003516886428136976, "bike": 0.003516886428136976}]
null	["001", "2008-10-23 01:53:15", 39.984686, 116.318417, "NA", {"all": 0.0}]
null	["001", "2008-10-23 02:53:20", 39.984688, 116.318385, "taxi", {"all": 0.0}]
null	["001", "2008-10-23 02:53:50", 39.984606, 116.317065, "taxi", {"taxi": 0.11283223062896626, "all": 0.11283223062896626}]
null	["002", "2008-10-23 02:51:25", 39.984655, 116.318263, "taxi", {"all": 0.0}]
null	["002", "2008-10-23 02:53:15", 39.984539, 116.317294, "NA", {"NA": 0.08355982534929753, "all": 0.08355982534929753}]
null	["002", "2008-10-23 02:53:50", 39.984606, 116.317065, "taxi", {"taxi": 0.020884688774914933, "NA": 0.08355982534929753, "all": 0.10444451412421246}]


#Main Aggregation MrJob

In [168]:
%%writefile Aggregate.py

from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.compat import jobconf_from_env
from datetime import datetime
from datetime import timedelta
import ast
import math
# from mrjob.protocol import JSONValueProtocol

sort_conf = {
    'mapred.reduce.tasks': '1'
}

class Aggregate(MRJob):

#     INPUT_PROTOCOL = JSONValueProtocol
    
    def steps(self):
        return [MRStep(mapper_init=self.mapper1_init,
                       mapper=self.mapper1,
                       reducer_init=self.reducer1_init,
                       reducer=self.reducer1,
                       reducer_final=self.reducer1_final
                      )
                ,
                MRStep(mapper=self.mapper2
                       ,
                       reducer_init=self.reducer2_init,
                       reducer=self.reducer2,
                       reducer_final=self.reducer2_final),
                MRStep(mapper=self.mapper3,
                       reducer=self.reducer3,
                       jobconf=sort_conf
                      )
               ]
    
    ### First MrJob: aggregate by the time interval
    
    
    def mapper1_init(self):
        self.interval = int(jobconf_from_env('interval'))
    
    def mapper1(self, _, line):
        key,data = line.strip().split("\t")
        
        user_id,dt,x_new,y_new,transport,dist = ast.literal_eval(data)
        dt_convert = datetime.strptime(dt, '%Y-%m-%d %H:%M:%S')
        sec = dt_convert.second
        remainder = sec%self.interval
        if remainder != 0:  
            update_sec = self.interval-remainder
        else:
            update_sec=0
        
        dt_convert += timedelta(0,update_sec)
        dt_convert = dt_convert.strftime("%Y-%m-%d %H:%M:%S")
        yield (dt_convert,user_id), (dt,x_new,y_new,transport,dist)
    
    def reducer1_init(self):
        self.current_dt = ''
        self.current_user=''
        self.current_max_dist = 0.0
        self.aggregate = {}

    
    def reducer1(self, key, values):
        dt_convert,user_id = key
        self.aggregate.setdefault(dt_convert,{})
        
        if dt_convert != self.current_dt or user_id != self.current_user:
            self.current_dt = dt_convert
            self.current_user = user_id
            self.current_max_dist = 0.0

        for value in values:
            dt,x_new,y_new,transport,dist = value
            if dist['all'] >= self.current_max_dist :
                self.current_max_dist = dist['all']
#                 value.append(user_id)
                self.aggregate[dt_convert][user_id]=value
        
    def reducer1_final(self):
        for dt_convert, values in self.aggregate.iteritems():
            for user_id, value in values.iteritems(): 
                dt,x_new,y_new,transport,dist = value
                yield dt_convert, (user_id,dt,x_new,y_new,transport,dist)
        
    ### 2nd MrJob: aggregate by the users within the time interval
   
    def mapper2(self, dt_convert, value):
        yield dt_convert, value
        
    def reducer2_init(self):
        self.current_dt = ''
        self.aggregate = {}
#         self.users={}
#         self.transports={}
        self.dists={}
            
    def reducer2(self, dt_convert, values): 
        
        for value in values:
            user_id,dt,x_new,y_new,transport,dists = value
            self.aggregate.setdefault(dt_convert,{})

            self.aggregate[dt_convert].setdefault("users",[])
            self.aggregate[dt_convert]["users"].append({user_id:[x_new,y_new,transport]})

            self.aggregate[dt_convert].setdefault("transports",{})
            self.aggregate[dt_convert]["transports"].setdefault(transport,0)
            self.aggregate[dt_convert]["transports"][transport]+=1

            self.dists[user_id] = dists
#             for transport_mode,dist in dists.iteritems():
#                 self.dists.setdefault(user_id,{})
#                 self.dists[user_id].setdefault(transport_mode,0.0)
#                 self.dists[user_id][transport_mode] += dist
#                 self.dists[user]
        
        for user_id, dists in self.dists.iteritems():
            self.aggregate[dt_convert].setdefault('dists',{})
            for transport_mode, dist in dists.iteritems():
                self.aggregate[dt_convert]['dists'].setdefault(transport_mode,0.0)
                self.aggregate[dt_convert]['dists'][transport_mode] += dist
            
#             self.aggregate[dt_convert]['dists'] = self.dists.copy()
                
    def reducer2_final(self):                         
        for dt_convert, value in self.aggregate.iteritems():
            yield dt_convert, value

### 3rd MrJob: sort by dt_convert
    
    def mapper3(self,dt_convert,value):
        yield dt_convert,value
        
    def reducer3(self,dt_convert,values):
        for value in values:
            yield dt_convert,value
            
if __name__ == '__main__':
    Aggregate.run()


Overwriting Aggregate.py


In [68]:
#create new job flow (small cluster)
!python /Users/maktrix/Dropbox/Berkeley/W261_ML_scale/mrjob/mrjob/tools/emr/create_job_flow.py

using configs in /Users/maktrix/.mrjob.conf
Creating persistent job flow to run several jobs in...
creating tmp directory /var/folders/s3/ll81spln3fn418tf8hhv248m0000gn/T/no_script.maktrix.20151107.134656.227702
writing master bootstrap script to /var/folders/s3/ll81spln3fn418tf8hhv248m0000gn/T/no_script.maktrix.20151107.134656.227702/b.py
Copying non-input files into s3://w209-d3/no_script.maktrix.20151107.134656.227702/files/
Waiting 5.0s for S3 eventual consistency
Creating Elastic MapReduce job flow
Job flow created with ID: j-1JE76Q6E0IZLL
j-1JE76Q6E0IZLL


In [181]:
%reload_ext autoreload
%autoreload 2

import os

def aggregate_driver(inputfolder='', outputfolder='', onEMR=False, job_flow_id=''):

    from Aggregate import Aggregate
    
    if onEMR:
        os.system("aws s3 rm "+inputfolder+"_SUCCESS")
        os.system("aws s3 rm --recursive "+outputfolder)
        mr_job = Aggregate(args=[inputfolder,
                                 '-r','emr',
                                 '--output-dir=' + outputfolder,
#                                  '--emr-job-flow-id=' + job_flow_id,
                                 '--jobconf', 'interval=15'])    
    else:
        !rm -r ../../Geolife_Trajectories_1.3/local/aggregate/
        mr_job = Aggregate(args=[inputfolder,
                                 '--output-dir=' + '../../Geolife_Trajectories_1.3/local/aggregate/',
                                 '--jobconf', 'interval=15'])
        
    with mr_job.make_runner() as runner:
        runner.run()

    
# run driver function (EMR)
aggregate_driver(inputfolder='s3://w209-d3/calc-dist2/data_csv/', 
             outputfolder='s3://w209-d3/aggregate2/data_csv/',
             onEMR=True, 
             job_flow_id='')
!rm -r ../../Geolife_Trajectories_1.3/local/aggregate2/
!mkdir ../../Geolife_Trajectories_1.3/local/aggregate2/
!aws s3 cp s3://w209-d3/aggregate2/data_csv/part-00000 ../../Geolife_Trajectories_1.3/local/aggregate2/part-00000

    
## run driver function (locally)
# aggregate_driver(inputfolder='../../Geolife_Trajectories_1.3/local/calc-dist/')  


download: s3://w209-d3/aggregate2/data_csv/part-00000 to ../../Geolife_Trajectories_1.3/local/aggregate2/part-00000


In [182]:
!tail ../../Geolife_Trajectories_1.3/local/aggregate2/*

"2012-07-27 08:29:15"	{"transports": {"NA": 1}, "users": [{"163": [39.984516663000001, 116.302059985, "NA"]}], "dists": {"taxi": 4440.6102291981524, "bike": 6309.3140097796568, "all": 1127045.3598806493, "run": 5.7338214379781771, "car": 25881.033540941578, "bus": 18849.715606401976, "walk": 6880.4236643397899, "train": 34263.540181645367, "motorcycle": 4.303923491750826, "NA": 1011025.895265318, "airplane": 11837.318163599522, "boat": 146.86584636220212, "subway": 7400.6056281401061}}
"2012-07-27 08:29:30"	{"transports": {"NA": 1}, "users": [{"163": [39.983167424000001, 116.302093149, "NA"]}], "dists": {"taxi": 4440.6102291981524, "bike": 6309.3140097796568, "all": 1127045.5099743095, "run": 5.7338214379781771, "car": 25881.033540941578, "bus": 18849.715606401976, "walk": 6880.4236643397899, "train": 34263.540181645367, "motorcycle": 4.303923491750826, "NA": 1011026.0453589781, "airplane": 11837.318163599522, "boat": 146.86584636220212, "subway": 7400.6056281401061}}
"2012-07-27 08:29

In [183]:
##EMR RESULTS
# import smart_open
# index = 0
# filename='s3://w209-d3/aggregate/data_csv/part-00000'
# for line in smart_open.smart_open(filename):
#     print line.strip().split("\t")
#     index +=1
#     if index==10000: break

##LOCAL RESULTS
# !head -n100000 ../../Geolife_Trajectories_1.3/local/aggregate/*
!tail ../../Geolife_Trajectories_1.3/local/aggregate3/*


"2012-07-27 08:29:15"	{"transports": {"NA": 1}, "users": [{"163": [39.984516663000001, 116.302059985, "NA"]}], "dists": {"taxi": 4196.7528788593245, "bike": 6205.3158435967216, "all": 1077668.9341454383, "run": 5.6461644826862152, "car": 23786.261057368167, "bus": 18029.82053246229, "walk": 6040.8616079211179, "train": 29149.117909132881, "motorcycle": 4.303923491750826, "NA": 972303.29654944164, "airplane": 11447.165041356469, "boat": 146.78578788456349, "subway": 6353.6068494493229}}
"2012-07-27 08:29:30"	{"transports": {"NA": 1}, "users": [{"163": [39.983167424000001, 116.302093149, "NA"]}], "dists": {"taxi": 4196.7528788593245, "bike": 6205.3158435967216, "all": 1077669.0842390985, "run": 5.6461644826862152, "car": 23786.261057368167, "bus": 18029.82053246229, "walk": 6040.8616079211179, "train": 29149.117909132881, "motorcycle": 4.303923491750826, "NA": 972303.44664310187, "airplane": 11447.165041356469, "boat": 146.78578788456349, "subway": 6353.6068494493229}}
"2012-07-27 08:29:

#Summary Statistics


In [171]:
!aws s3 cp --recursive s3://w209-d3/aggregate3/  ../../Geolife_Trajectories_1.3/aggregate3/

download: s3://w209-d3/aggregate3/data_csv/_SUCCESS to ../../Geolife_Trajectories_1.3/aggregate3/data_csv/_SUCCESS
^CCleaning up. Please wait...                         



In [152]:
!mv ../../Geolife_Trajectories_1.3/aggregate3/ ../../Geolife_Trajectories_1.3/local/aggregate3/